Credit to https://github.com/harvitronix/neural-network-genetic-algorithm

Data set

In [ ]:
from keras.datasets import mnist
import numpy as np
from numpy import isneginf
from scipy.stats import iqr 
(X_train1, y_train1), (X_test1, y_test1) = mnist.load_data()


def compute_mag_phase(toBeTransfromed):
    dft = np.fft.fft2(toBeTransfromed)
    dft_shift = np.fft.fftshift(dft)
    mag = np.abs(dft_shift)
    ang = np.angle(dft_shift)
    return mag, ang

def reconstruct(mag,ang):
    combined = np.multiply(mag, np.exp(1j*ang))
    #combinedReal = np.real(combined)
    #combinedImag = np.imag(combined)
    fftx = np.fft.ifftshift(combined)
    ffty = np.fft.ifft2(fftx)
    imgCombined = np.abs(ffty)
    return imgCombined

def pre_process_mag(dataset):
    mag = np.zeros(dataset.shape)
    phase = np.zeros(dataset.shape)
    x= 0
    for i in range(0,dataset.shape[0]):
      mag[i,:,:],phase[i,:,:] = compute_mag_phase(dataset[i])
    return mag,phase

X_train_mag,X_train_phase = pre_process_mag(X_train1)
X_test_mag,X_test_phase = pre_process_mag(X_test1)

# flatten the magnitude
X_train_mag_flatten = X_train_mag.reshape(X_train_mag.shape[0], X_train_phase.shape[1] * X_train_mag.shape[2])
X_test_mag_flatten = X_test_mag.reshape(X_test_mag.shape[0], X_test_mag.shape[1] * X_test_mag.shape[2])

# flatten the phase
X_train_phase_flatten = X_train_phase.reshape(X_train_phase.shape[0], X_train_phase.shape[1] * X_train_phase.shape[2])
X_test_phase_flatten = X_test_phase.reshape(X_test_phase.shape[0], X_test_phase.shape[1] * X_test_phase.shape[2])


X_train_mag_flatten = X_train_mag_flatten.astype('float32')
X_test_mag_flatten = X_test_mag_flatten.astype('float32')

X_s = np.concatenate((X_train_mag_flatten,X_test_mag_flatten),axis =0)
IQR = iqr(X_s)
Median = np.median(X_s)

X_train_m_scaled = (X_train_mag_flatten - Median) / IQR
X_test_m_scaled = (X_test_mag_flatten - Median) / IQR



Training

In [ ]:
from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Helper: Early stopping.
early_stopper = EarlyStopping(patience=5)

def get_mnist():
    """Retrieve the MNIST dataset and process the data."""
    # Set defaults.
    nb_classes = 10
    batch_size = 128
    input_shape = (784,)

    # Get the data.
    x_train = X_train_m_scaled
    x_test = X_test_m_scaled

    # convert class vectors to binary class matrices
    y_train = to_categorical(y_train1, nb_classes)
    y_test = to_categorical(y_test1, nb_classes)
    print('done mnist')
    return (nb_classes, batch_size, input_shape, x_train, x_test, y_train, y_test)

def compile_model(network, nb_classes, input_shape):
    """Compile a sequential model.
    Args:
        network (dict): the parameters of the network
    Returns:
        a compiled network.
    """
    # Get our network parameters.
    nb_layers = network['nb_layers']
    nb_neurons1 = network['nb_neurons1']
    nb_neurons2 = network['nb_neurons2']
    nb_neurons3 = network['nb_neurons3']
    nb_neurons4 = network['nb_neurons4']
    activation = network['activation']
    optimizer = network['optimizer']

    model = Sequential()

    # Add each layer.
    for i in range(nb_layers):

        # Need input shape for first layer.
        if i == 0:
            model.add(Dense(nb_neurons1, activation=activation, input_shape=input_shape))
        elif i == 1:
            model.add(Dense(nb_neurons2, activation=activation))
        elif i == 2:
            model.add(Dense(nb_neurons3, activation=activation))
        elif i == 3:
            model.add(Dense(nb_neurons4, activation=activation))

        model.add(Dropout(0.2))  # hard-coded dropout

    # Output layer.
    model.add(Dense(nb_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                  metrics=['accuracy'])
    print('compiled model')
    return model

def train_and_score(network, dataset):
    """Train the model, return test loss.
    Args:
        network (dict): the parameters of the network
        dataset (str): Dataset to use for training/evaluating
    """
    logging.info("***train_and_score***")
    if dataset == 'cifar10':
        nb_classes, batch_size, input_shape, x_train, \
            x_test, y_train, y_test = get_cifar10()
    elif dataset == 'mnist':
        logging.info("***mnist***")
        nb_classes, batch_size, input_shape, x_train, \
            x_test, y_train, y_test = get_mnist()

    model = compile_model(network, nb_classes, input_shape)
    logging.info("***compiled_model***")
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=100,  # using early stopping, so no real limit
              verbose=0,
              validation_split=0.2,
              callbacks=[early_stopper])
    #validation_split=0.2, epochs=100, batch_size=2000,
    #validation_data=(x_test, y_test),
    score = model.evaluate(x_test, y_test, verbose=0)
    logging.info("***trainedd***")
    print(score[1])
    return score[1]  # 1 is accuracy. 0 is loss.

Network

In [ ]:
import random
import logging
#from train import train_and_score

class Network():
    """Represent a network and let us operate on it.
    Currently only works for an MLP.
    """

    def __init__(self, nn_param_choices=None):
        """Initialize our network.
        Args:
            nn_param_choices (dict): Parameters for the network, includes:
                nb_neurons (list): [64, 128, 256]
                nb_layers (list): [1, 2, 3, 4]
                activation (list): ['relu', 'elu']
                optimizer (list): ['rmsprop', 'adam']
        """
        self.accuracy = 0.
        self.nn_param_choices = nn_param_choices
        self.network = {}  # (dic): represents MLP network parameters

    def create_random(self):
        i = True 
        """Create a random network."""
        for key in self.nn_param_choices:
            if key == 'nb_neurons2':
              while (True):
                logging.info("stuck 1")
                self.network[key] = random.choice(self.nn_param_choices[key])
                if self.network[key] < self.network['nb_neurons1']:
                  break
            elif key == 'nb_neurons3':
              while (True):
                logging.info("stuck 2")
                self.network[key] = random.choice(self.nn_param_choices[key])
                if self.network[key] < self.network['nb_neurons2']:
                  break
            elif key == 'nb_neurons4':
              while (True):
                logging.info("stuck 3")
                self.network[key] = random.choice(self.nn_param_choices[key])
                if self.network[key] < self.network['nb_neurons3']:
                  break
            else:
              self.network[key] = random.choice(self.nn_param_choices[key])



    def create_set(self, network):
        """Set network properties.
        Args:
            network (dict): The network parameters
        """
        self.network = network

    def train(self, dataset):
        """Train the network and record the accuracy.
        Args:
            dataset (str): Name of dataset to use.
        """
        logging.info("***train***")
        if self.accuracy == 0.:
            self.accuracy = train_and_score(self.network, dataset)

    def print_network(self):
        """Print out a network."""
        logging.info(self.network)
        logging.info("Network accuracy: %.2f%%" % (self.accuracy * 100))

In [ ]:
from functools import reduce
from operator import add
import random


class Optimizer():
    """Class that implements genetic algorithm for MLP optimization."""

    def __init__(self, nn_param_choices, retain=0.4,
                 random_select=0.1, mutate_chance=0.3):
        """Create an optimizer.
        Args:
            nn_param_choices (dict): Possible network paremters
            retain (float): Percentage of population to retain after
                each generation
            random_select (float): Probability of a rejected network
                remaining in the population
            mutate_chance (float): Probability a network will be
                randomly mutated
        """
        self.mutate_chance = mutate_chance
        self.random_select = random_select
        self.retain = retain
        self.nn_param_choices = nn_param_choices

    def create_population(self, count):
        """Create a population of random networks.
        Args:
            count (int): Number of networks to generate, aka the
                size of the population
        Returns:
            (list): Population of network objects
        """
        pop = []
        for _ in range(0, count):
            # Create a random network.
            network = Network(self.nn_param_choices)
            logging.info("count")
            network.create_random()

            # Add the network to our population.
            pop.append(network)

        return pop

    @staticmethod
    def fitness(network):
        """Return the accuracy, which is our fitness function."""
        return network.accuracy

    def grade(self, pop):
        """Find average fitness for a population.
        Args:
            pop (list): The population of networks
        Returns:
            (float): The average accuracy of the population
        """
        summed = reduce(add, (self.fitness(network) for network in pop))
        return summed / float((len(pop)))

    def breed(self, mother, father):
        """Make two children as parts of their parents.
        Args:
            mother (dict): Network parameters
            father (dict): Network parameters
        Returns:
            (list): Two network objects
        """
        children = []
        for _ in range(2):

            child = {}

            #  need to do if they have equal layers
            # Loop through the parameters and pick params for the kid.
            for param in self.nn_param_choices:
                  if (param == 'nb_neurons2') :
                    while (True):
                        child[param] = random.choice([mother.network[param], father.network[param]])
                        if child[param] < child['nb_neurons1']:
                            logging.info("***broke1***")
                            break
                  elif param == 'nb_neurons3':
                    while ( True):
                        child[param] = random.choice([mother.network[param], father.network[param]])
                        if child[param] < child['nb_neurons2']:
                            break
                            logging.info("***broke2***")
                  elif param == 'nb_neurons4':
                    while (True):
                        child[param] = random.choice([mother.network[param], father.network[param]])
                        if child[param] < child['nb_neurons3']:
                            break
                            logging.info("***broke3***")
                  else:
                    child[param] = random.choice([mother.network[param], father.network[param]])

            # Now create a network object.
            network = Network(self.nn_param_choices)
            network.create_set(child)

            # Randomly mutate some of the children.
            if self.mutate_chance > random.random():
                network = self.mutate(network)

            children.append(network)

        return children

    def mutate(self, network):
        """Randomly mutate one part of the network.
        Args:
            network (dict): The network parameters to mutate
        Returns:
            (Network): A randomly mutated network object
        """
        # Choose a random key.

 
        mutation = random.choice(list(self.nn_param_choices.keys()))
        
        #mutate param within condition
        if mutation == 'nb_neurons1':
            while (True):
                  network.network[mutation] = random.choice(self.nn_param_choices[mutation])
                  if (network.network[mutation] > network.network['nb_neurons2']):
                    break
        elif mutation == 'nb_neurons2':
            while (True):
                  network.network[mutation] = random.choice(self.nn_param_choices[mutation])
                  if (network.network[mutation] > network.network['nb_neurons3']) and (network.network[mutation] < network.network['nb_neurons1']) :
                    break
        elif mutation == 'nb_neurons3':
            while (True):
                  network.network[mutation] = random.choice(self.nn_param_choices[mutation])
                  if (network.network[mutation] > network.network['nb_neurons4'])  and (network.network[mutation] < network.network['nb_neurons2']):
                    break
        elif mutation == 'nb_neurons4':
            while (True):
                  network.network[mutation] = random.choice(self.nn_param_choices[mutation])
                  if (network.network[mutation] < network.network['nb_neurons3']):
                    break
        else:
          # Mutate one of the params.
          network.network[mutation] = random.choice(self.nn_param_choices[mutation])
        return network

    def evolve(self, pop):
        """Evolve a population of networks.
        Args:
            pop (list): A list of network parameters
        Returns:
            (list): The evolved population of networks
        """
        # Get scores for each network.
        graded = [(self.fitness(network), network) for network in pop]

        # Sort on the scores.
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]

        # Get the number we want to keep for the next gen.
        retain_length = int(len(graded)*self.retain)

        # The parents are every network we want to keep.
        parents = graded[:retain_length]

        # For those we aren't keeping, randomly keep some anyway.
        for individual in graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        # Now find out how many spots we have left to fill.
        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []

        # Add children, which are bred from two remaining networks.
        while len(children) < desired_length:

            # Get a random mom and dad.
            print(parents_length-1)
            male = random.randint(0, parents_length-1)

            female = random.randint(0, parents_length-1)

            # Assuming they aren't the same network...
            if male != female:
                male = parents[male]
                female = parents[female]

                # Breed them.
                babies = self.breed(male, female)

                # Add the children one at a time.
                for baby in babies:
                    # Don't grow larger than desired length.
                    if len(children) < desired_length:
                        children.append(baby)

        parents.extend(children)

        return parents

In [ ]:
import logging
from tqdm import tqdm

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt'
)

def train_networks(networks, dataset):
    """Train each network.
    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """

    pbar = tqdm(total=len(networks))
    logging.info("***a***")
    for network in networks:
        network.train(dataset)
        pbar.update(1)
        logging.info("***b***")
    pbar.close()

def get_average_accuracy(networks):
    """Get the average accuracy for a group of networks.
    Args:
        networks (list): List of networks
    Returns:
        float: The average accuracy of a population of networks.
    """
    total_accuracy = 0
    for network in networks:
        total_accuracy += network.accuracy

    return total_accuracy / len(networks)

def generate(generations, population, nn_param_choices, dataset):
    """Generate a network with the genetic algorithm.
    Args:
        generations (int): Number of times to evole the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating
    """
    optimizer = Optimizer(nn_param_choices)
    logging.info("***Optimizer***")
    networks = optimizer.create_population(population)

    # Evolve the generation.
    for i in range(generations):
        logging.info("***Doing generation %d of %d***" %
                     (i + 1, generations))

        # Train and get accuracy for networks.
        train_networks(networks, dataset)
        logging.info("***Trained networks***")
        # Get the average accuracy for this generation.
        average_accuracy = get_average_accuracy(networks)

        # Print out the average accuracy each generation.
        logging.info("Generation average: %.2f%%" % (average_accuracy * 100))
        logging.info('-'*80)

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)

    # Sort our final population.
    networks = sorted(networks, key=lambda x: x.accuracy, reverse=True)

    # Print out the top 5 networks.
    print_networks(networks[:5])

def print_networks(networks):
    """Print a list of networks.
    Args:
        networks (list): The population of networks
    """
    logging.info('-'*80)
    for network in networks:
        network.print_network()

def main():
    """Evolve a network."""
    generations = 7  # Number of times to evole the population.
    population = 30  # Number of networks in each generation.
    dataset = 'mnist'
    print(2)

    nn_param_choices = {
        #'nb_neurons': [10, 60, 110, 160, 210, 260,310,360,410,460,510],
        'nb_neurons1': range(14,780),
        'nb_neurons2':  range(13,779),
        'nb_neurons3': range(12,778),
        'nb_neurons4': range(11,777),
        'nb_layers': [1, 2, 3, 4],
        'activation': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }
    print(3)
    logging.info("***Evolving %d generations with population %d***" %
                 (generations, population))
    print(4)
    generate(generations, population, nn_param_choices, dataset)
    print('done')

if __name__ == '__main__':
    main()

  0%|          | 0/30 [00:00<?, ?it/s]

2
3
4
done mnist
compiled model


  3%|▎         | 1/30 [01:29<43:17, 89.56s/it]

0.9093999862670898
done mnist
compiled model


  7%|▋         | 2/30 [01:42<31:00, 66.45s/it]

...

0.9491999745368958
done mnist
compiled model


 97%|█████████▋| 29/30 [10:14<00:43, 43.87s/it]

0.9433000087738037
done mnist
compiled model


100%|██████████| 30/30 [11:02<00:00, 22.09s/it]

0.9445000290870667
done


{'nb_neurons1': 503, 'nb_neurons2': 500, 'nb_neurons3': 66, 'nb_neurons4': 48, 'nb_layers': 3, 'activation': 'relu', 'optimizer': 'sgd'}
Network accuracy: 95.06%

{'nb_neurons1': 503, 'nb_neurons2': 500, 'nb_neurons3': 66, 'nb_neurons4': 48, 'nb_layers': 3, 'activation': 'elu', 'optimizer': 'sgd'}
Network accuracy: 94.94%

{'nb_neurons1': 688, 'nb_neurons2': 310, 'nb_neurons3': 96, 'nb_neurons4': 29, 'nb_layers': 3, 'activation': 'relu', 'optimizer': 'sgd'}
Network accuracy: 94.92%

{'nb_neurons1': 407, 'nb_neurons2': 310, 'nb_neurons3': 182, 'nb_neurons4': 130, 'nb_layers': 4, 'activation': 'elu', 'optimizer': 'sgd'}
Network accuracy: 94.89%

{'nb_neurons1': 639, 'nb_neurons2': 500, 'nb_neurons3': 66, 'nb_neurons4': 48, 'nb_layers': 3, 'activation': 'relu', 'optimizer': 'sgd'}
Network accuracy: 94.80%

Potential to add starting points.